<a href="https://colab.research.google.com/github/cagBRT/computer-vision/blob/master/pedestrians.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -l -s https://github.com/opencv/opencv.git 
!pwd

In [ ]:
!git clone -l -s https://github.com/cagBRT/computer-vision.git cloned-repo
%cd cloned-repo
!ls

In [ ]:
!pip install --upgrade imutils

In [ ]:
# import the necessary packages
from __future__ import print_function
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from imutils import paths
from imutils.object_detection import non_max_suppression

initialize the pedestrian detector. First, we make a call to hog = cv2.HOGDescriptor()  which initializes the Histogram of Oriented Gradients descriptor. Then, we call the setSVMDetector  to set the Support Vector Machine to be pre-trained pedestrian detector, loaded via the cv2.HOGDescriptor_getDefaultPeopleDetector()  function.

In [ ]:
# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [ ]:
image = cv2.imread("images/family.jpg")
cv2_imshow(image)

In [ ]:
imagePath = "images/family.jpg"
image = cv2.imread(imagePath)

**loading our image off disk and resizing it to have a maximum width of 400 pixels**. The reason we attempt to reduce our image dimensions is two-fold:

1. **Reducing image size ensures that less sliding windows in the image pyramid need to be evaluated** (i.e., have HOG features extracted from and then passed on to the Linear SVM), thus reducing detection time (and increasing overall detection throughput).<br>
2. **Resizing our image also improves the overall accuracy of our pedestrian detection** (i.e., less false-positives).

In [ ]:
#Resize
scale_percent = 80 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
orig = image.copy()
# resize image
image = cv2_imshow(image)

In [ ]:

	# detect people in the image
(rects, weights) = hog.detectMultiScale(image, 
                                        winStride=(4, 4),
                                        padding=(8, 8), 
                                        scale=1.01)

	# draw the original bounding boxes
for (x, y, w, h) in rects:
		cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)
	# apply non-maxima suppression to the bounding boxes using a
	# fairly large overlap threshold to try to maintain overlapping
	# boxes that are still people
rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
	# draw the final bounding boxes
for (xA, yA, xB, yB) in pick:
	cv2.rectangle(image, (xA, yA), (xB, yB), (0, 255, 0), 2)
	# show some information on the number of bounding boxes
filename = imagePath[imagePath.rfind("/") + 1:]
print("[INFO] {}: {} original boxes, {} after suppression".format(
		filename, len(rects), len(pick)))
	# show the output images
cv2_imshow(orig)
cv2_imshow( image)